In [1]:
import torch
import pyro
from pyro.distributions import Normal, Bernoulli

from causal_pyro.query.ops import expectation
from causal_pyro.query.handlers import MonteCarloIntegration

## Expectation


In [11]:
stress_pt = torch.tensor([0.5])
smokes_cpt = torch.tensor([0.2, 0.8])
cancer_cpt = torch.tensor([[0.1, 0.15], [0.8, 0.85]])
probs = (stress_pt, smokes_cpt, cancer_cpt)

def smoking_model(stress_pt, smokes_cpt, cancer_cpt):
    stress = pyro.sample("stress", Bernoulli(stress_pt)).long()
    smokes = pyro.sample(
        "smokes", Bernoulli(smokes_cpt[stress])
    ).long()
    cancer = pyro.sample(
        "cancer", Bernoulli(cancer_cpt[stress, smokes])
    ).long()
    return cancer

with MonteCarloIntegration(1000, parallel=True):
    result_cancer = expectation(smoking_model, "cancer", 0, probs, {})

print(result_cancer)  # exact value: .47

tensor([0.4830])
